In [1]:
import os, pathlib

root = pathlib.Path("evat-sprint4")
dirs = [
    root/"data",
    root/"routing_engine",
    root/"api",
]
for d in dirs:
    d.mkdir(parents=True, exist_ok=True)

# create empty files
for fp in [
    root/"routing_engine/__init__.py",
    root/"routing_engine/utils.py",
    root/"routing_engine/adapters.py",
    root/"routing_engine/engine.py",
    root/"api/main.py",
]:
    fp.touch(exist_ok=True)

print("Project created at:", root.resolve())


Project created at: C:\Users\Uday\evat-sprint4


In [2]:
%pip install fastapi "uvicorn[standard]" pydantic pandas numpy scikit-learn geopy requests



  Attempting uninstall: anyio
    Found existing installation: anyio 3.5.0
    Uninstalling anyio-3.5.0:
      Successfully uninstalled anyio-3.5.0



ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-server 1.18.1 requires anyio<4,>=3.1.0, but you have anyio 4.10.0 which is incompatible.


In [5]:
import shutil, pathlib

src = pathlib.Path("C:/Users/Uday/Downloads/Processed_Combined_Weather_Traffic_EV_with_Score.csv")
dst = pathlib.Path("evat-sprint4/data/combined_data.csv")
if src.exists():
    shutil.copy(src, dst)
    print("Copied to:", dst.resolve())
else:
    print("⚠️ Update 'src' to point to your dataset, or upload manually to:", dst)


Copied to: C:\Users\Uday\evat-sprint4\data\combined_data.csv


In [6]:
import pandas as pd
df = pd.read_csv("evat-sprint4/data/combined_data.csv", nrows=5)
df.head()


,InfrastructureID,RatePerHour,MaxTime,Latitude,Longitude,TAVG,PRCP,TMAX,TMIN,OBJECTID,...,PDF,geometry.2,geometry.3,Temp_Range,High_Temp_Flag,Low_Temp_Flag,No_Precip_Flag,Temp_Range_Score,Weather_Sensitivity_Score,Weather_Sensitivity_Score_Norm
0,11,4,-1,48.424600,-123.361093,26.3,0.0,38.4,18.2,19,...,NaN,14,26,20.2,1,0,1,1.0,3.0,0.6
1,18,2,-1,48.446989,-123.334368,20.9,0.0,32.6,19.9,197,...,NaN,8,24,12.7,0,0,1,0.5,0.5,0.1
2,20,2,-1,48.442372,-123.331238,17.3,0.0,21.8,15.3,430,...,NaN,6,81,6.5,0,0,1,0.0,0.0,0.0
3,2,1,2,48.422837,-123.367715,19.6,0.0,22.9,14.5,529,...,NaN,23,93,8.4,0,0,1,0.0,0.0,0.0
4,7,1,1,48.429492,-123.368619,22.2,0.0,30.1,15.8,778,...,NaN,27,91,14.3,0,0,1,0.5,0.5,0.1


In [7]:
%%writefile evat-sprint4/routing_engine/utils.py
import numpy as np
def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0088
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return 2*R*np.arcsin(np.sqrt(a))
def minmax_norm(series):
    s = series.astype(float)
    lo, hi = np.nanmin(s), np.nanmax(s)
    if not np.isfinite(hi - lo) or (hi - lo) == 0:
        return np.zeros_like(s, dtype=float)
    return (s - lo) / (hi - lo)


Overwriting evat-sprint4/routing_engine/utils.py


In [8]:
%%writefile evat-sprint4/routing_engine/adapters.py
import time
from functools import lru_cache
def _cache_key(lat, lon, minutes=5):
    bucket = int(time.time() // (60 * minutes))
    return (round(lat, 4), round(lon, 4), bucket)
@lru_cache(maxsize=10000)
def _cached_value(kind, key, default):
    return default
def get_weather_score(lat, lon, mode="FALLBACK", default=0.2):
    key = ("weather",) + _cache_key(lat, lon)
    return _cached_value("weather", key, default)
def get_traffic_proxy(lat, lon, mode="FALLBACK", default=0.2):
    key = ("traffic",) + _cache_key(lat, lon)
    return _cached_value("traffic", key, default)


Overwriting evat-sprint4/routing_engine/adapters.py


In [9]:
import importlib, sys
sys.path.append("evat-sprint4") 
utils = importlib.import_module("routing_engine.utils")
adapters = importlib.import_module("routing_engine.adapters")
print("Loaded:", utils.__name__, adapters.__name__)


Loaded: routing_engine.utils routing_engine.adapters


In [10]:
%%writefile evat-sprint4/routing_engine/engine.py
import pandas as pd
import numpy as np
from collections import defaultdict
import heapq
from .utils import haversine_km, minmax_norm
from .adapters import get_weather_score, get_traffic_proxy

DEFAULTS = {
    "EV_RANGE_KM": 35.0, "K_NEIGHBORS": 6, "ASSUMED_SPEED_KMH": 60.0,
    "ALPHA_WEATHER": 0.15, "BETA_TRAFFIC": 0.10, "CHARGE_TIME_MIN": 15.0,
    "MODE": "FALLBACK",
}

REQ_COLS = {
    "id": ["InfrastructureID","SiteID","StationID","ID","id"],
    "lat": ["Latitude","lat","LAT","Lat"],
    "lon": ["Longitude","lon","LON","Long","Lng"]
}

def pick(df, names):
    for n in names:
        if n in df.columns: return n
    return None

def load_dataset(path):
    df = pd.read_csv(path)

    # Weather score (if missing)
    if "Weather_Sensitivity_Score" not in df.columns:
        for c in ("TMAX","TMIN","PRCP"):
            if c not in df.columns: df[c] = 0.0
        df["Temp_Range"] = df.get("Temp_Range", df["TMAX"] - df["TMIN"])
        df["High_Temp_Flag"] = (pd.to_numeric(df["TMAX"], errors="coerce") > 35).astype(int)
        df["Low_Temp_Flag"]  = (pd.to_numeric(df["TMIN"], errors="coerce") < 5).astype(int)
        df["No_Precip_Flag"] = (pd.to_numeric(df["PRCP"], errors="coerce") == 0).astype(int)
        def tr_score(x):
            if pd.isna(x): return 0.5
            if x > 20: return 1.0
            if x < 10: return 0.0
            return 0.5
        df["Temp_Range_Score"] = pd.to_numeric(df["Temp_Range"], errors="coerce").apply(tr_score)
        df["Weather_Sensitivity_Score"] = (
            2.0*df["High_Temp_Flag"] + 1.5*df["Low_Temp_Flag"] + (1 - df["No_Precip_Flag"]) + df["Temp_Range_Score"]
        )
    df["Weather_Score_Norm"] = np.clip(pd.to_numeric(df["Weather_Sensitivity_Score"], errors="coerce")/5.0,0,1)

    # Traffic proxy (robust)
    if "Traffic_Proxy" not in df.columns:
        if "Congestion_Factor" in df.columns:
            df["Traffic_Proxy"] = pd.to_numeric(df["Congestion_Factor"], errors="coerce")
        elif "Traffic_Volume" in df.columns and "Traffic_Length" in df.columns:
            vol = pd.to_numeric(df["Traffic_Volume"], errors="coerce")
            seg = pd.to_numeric(df["Traffic_Length"], errors="coerce").replace(0, np.nan)
            df["Traffic_Proxy"] = vol / seg
        else:
            df["Traffic_Proxy"] = 0.0
    df["Traffic_Proxy_Norm"] = minmax_norm(pd.to_numeric(df["Traffic_Proxy"], errors="coerce"))
    return df

def build_nodes(df):
    id_col  = pick(df, REQ_COLS["id"])
    lat_col = pick(df, REQ_COLS["lat"])
    lon_col = pick(df, REQ_COLS["lon"])
    if not lat_col or not lon_col:
        raise ValueError("Latitude/Longitude not found.")
    nodes = df[[id_col, lat_col, lon_col, "Weather_Score_Norm", "Traffic_Proxy_Norm"]].dropna().copy()
    nodes = nodes.drop_duplicates(subset=[id_col]).reset_index(drop=True)
    nodes.rename(columns={id_col:"node_id", lat_col:"lat", lon_col:"lon"}, inplace=True)
    return nodes

def build_edges(nodes, K_NEIGHBORS, EV_RANGE_KM):
    coords = nodes[["lat","lon"]].to_numpy()
    edges = []
    for i, (la, lo) in enumerate(coords):
        d = haversine_km(la, lo, coords[:,0], coords[:,1])
        idx = np.argsort(d)[1:K_NEIGHBORS+1]
        for j in idx:
            dist_km = float(d[j])
            if dist_km <= EV_RANGE_KM:
                edges.append((i, j, dist_km))
    return edges

def base_time_minutes(dist_km, speed_kmh):
    return 60.0 * dist_km / max(speed_kmh, 1e-6)

def build_costs(nodes, edges, speed_kmh, alpha, beta, mode="FALLBACK"):
    costs = []
    for (u, v, dist_km) in edges:
        travel_min = base_time_minutes(dist_km, speed_kmh)
        w = get_weather_score(nodes.loc[v,"lat"], nodes.loc[v,"lon"], mode=mode,
                              default=float(nodes.loc[v,"Weather_Score_Norm"]))
        t = get_traffic_proxy(nodes.loc[v,"lat"], nodes.loc[v,"lon"], mode=mode,
                              default=float(nodes.loc[v,"Traffic_Proxy_Norm"]))
        cost_min = travel_min * (1 + alpha*w + beta*t)
        costs.append((u, v, dist_km, travel_min, cost_min))
    return costs

def dijkstra_with_stop_penalty(nodes, costs, start_idx, goal_idx, charge_penalty_min):
    adj = defaultdict(list)
    for (u, v, dist_km, travel_min, cost_min) in costs:
        adj[u].append((v, cost_min, dist_km, travel_min))
    N = len(nodes)
    dist = [float('inf')]*N
    prev = [-1]*N
    dist[start_idx] = 0.0
    pq = [(0.0, start_idx)]
    visited = set()
    while pq:
        cur_cost, u = heapq.heappop(pq)
        if u in visited:
            continue
        visited.add(u)
        if u == goal_idx:
            break
        for v, edge_cost, _dkm, _tmin in adj[u]:
            add = charge_penalty_min if v != goal_idx else 0.0
            new_cost = cur_cost + edge_cost + add
            if new_cost < dist[v]:
                dist[v] = new_cost
                prev[v] = u
                heapq.heappush(pq, (new_cost, v))
    if dist[goal_idx] == float('inf'):
        return None, float('inf')
    path = []
    x = goal_idx
    while x != -1:
        path.append(x)
        x = prev[x]
    return path[::-1], dist[goal_idx]

def summarize_route(nodes, path):
    if not path: return []
    out = []
    for i, n in enumerate(path):
        out.append({
            "step": i,
            "node_index": int(n),
            "node_id": nodes.loc[n,"node_id"],
            "lat": float(nodes.loc[n,"lat"]),
            "lon": float(nodes.loc[n,"lon"]),
        })
    return out

def plan_route(
    data_path="data/combined_data.csv",
    start_idx=0, goal_idx=10,
    EV_RANGE_KM=DEFAULTS["EV_RANGE_KM"], K_NEIGHBORS=DEFAULTS["K_NEIGHBORS"],
    ASSUMED_SPEED_KMH=DEFAULTS["ASSUMED_SPEED_KMH"],
    ALPHA_WEATHER=DEFAULTS["ALPHA_WEATHER"], BETA_TRAFFIC=DEFAULTS["BETA_TRAFFIC"],
    CHARGE_TIME_MIN=DEFAULTS["CHARGE_TIME_MIN"], MODE=DEFAULTS["MODE"]
):
    df = load_dataset(data_path)
    nodes = build_nodes(df)
    edges = build_edges(nodes, K_NEIGHBORS, EV_RANGE_KM)
    costs = build_costs(nodes, edges, ASSUMED_SPEED_KMH, ALPHA_WEATHER, BETA_TRAFFIC, mode=MODE)
    path, total_cost = dijkstra_with_stop_penalty(nodes, costs, start_idx, goal_idx, CHARGE_TIME_MIN)
    return {
        "params": dict(EV_RANGE_KM=EV_RANGE_KM, K_NEIGHBORS=K_NEIGHBORS,
                       ASSUMED_SPEED_KMH=ASSUMED_SPEED_KMH,
                       ALPHA_WEATHER=ALPHA_WEATHER, BETA_TRAFFIC=BETA_TRAFFIC,
                       CHARGE_TIME_MIN=CHARGE_TIME_MIN, MODE=MODE),
        "nodes_count": len(nodes),
        "edges_count": len(edges),
        "path": summarize_route(nodes, path),
        "total_cost_min": float(total_cost)
    }


Overwriting evat-sprint4/routing_engine/engine.py


In [11]:
import sys, importlib
sys.path.append("evat-sprint4")
engine = importlib.import_module("routing_engine.engine")
print("Loaded:", engine.__name__)
print("Defaults:", engine.DEFAULTS)


Loaded: routing_engine.engine
Defaults: {'EV_RANGE_KM': 35.0, 'K_NEIGHBORS': 6, 'ASSUMED_SPEED_KMH': 60.0, 'ALPHA_WEATHER': 0.15, 'BETA_TRAFFIC': 0.1, 'CHARGE_TIME_MIN': 15.0, 'MODE': 'FALLBACK'}


In [12]:
%%writefile evat-sprint4/api/main.py
from fastapi import FastAPI
from pydantic import BaseModel
from routing_engine.engine import plan_route, load_dataset, build_nodes

app = FastAPI(title="EVAT Routing API")
DATA_PATH = "data/combined_data.csv"
_df = load_dataset(DATA_PATH)
_nodes = build_nodes(_df)

class RouteRequest(BaseModel):
    start_idx: int = 0
    goal_idx: int = 10
    ev_range_km: float = 35.0
    k_neighbors: int = 6
    assumed_speed_kmh: float = 60.0
    alpha_weather: float = 0.15
    beta_traffic: float = 0.10
    charge_penalty_min: float = 15.0
    mode: str = "FALLBACK"

@app.get("/health")
def health():
    return {"status": "ok", "nodes": len(_nodes)}

@app.get("/stations")
def stations():
    return _nodes.to_dict(orient="records")

@app.post("/route")
def route(req: RouteRequest):
    result = plan_route(
        data_path=DATA_PATH,
        start_idx=req.start_idx, goal_idx=req.goal_idx,
        EV_RANGE_KM=req.ev_range_km, K_NEIGHBORS=req.k_neighbors,
        ASSUMED_SPEED_KMH=req.assumed_speed_kmh,
        ALPHA_WEATHER=req.alpha_weather, BETA_TRAFFIC=req.beta_traffic,
        CHARGE_TIME_MIN=req.charge_penalty_min, MODE=req.mode
    )
    return result


Overwriting evat-sprint4/api/main.py


In [13]:
import subprocess, sys, time

proc = subprocess.Popen(
    [sys.executable, "-m", "uvicorn", "api.main:app", "--host", "127.0.0.1", "--port", "8000"],
    cwd="evat-sprint4",
)
time.sleep(2)
print("API started at http://127.0.0.1:8000  (PID:", proc.pid, ")")
print("Docs:       http://127.0.0.1:8000/docs")


API started at http://127.0.0.1:8000  (PID: 42064 )
Docs:       http://127.0.0.1:8000/docs


In [ ]:
import requests, json

payload = {
    "start_idx": 0,
    "goal_idx": min(10, max(1, len(stations)-1)),
    "ev_range_km": 35,
    "k_neighbors": 6,
    "assumed_speed_kmh": 60,
    "alpha_weather": 0.15,
    "beta_traffic": 0.10,
    "charge_penalty_min": 15,
    "mode": "FALLBACK"
}

r = requests.post("http://127.0.0.1:8000/route", json=payload)
print("status:", r.status_code)
print("content-type:", r.headers.get("content-type"))
print("raw response (first 800 chars):\n", r.text[:800])


In [16]:
try: proc.terminate()
except: pass

import subprocess, sys, time
proc = subprocess.Popen(
    [sys.executable, "-m", "uvicorn", "api.main:app", "--host", "127.0.0.1", "--port", "8000"],
    cwd="evat-sprint4",
)
time.sleep(2)


In [17]:
import requests
print(requests.get("http://127.0.0.1:8000/health").json())
print(len(requests.get("http://127.0.0.1:8000/stations").json()))


{'status': 'ok', 'nodes': 28}
28


In [18]:
import requests, json

# Quick check of valid index range (so you can pick start/goal confidently)
stations = requests.get("http://127.0.0.1:8000/stations").json()
print("Stations count:", len(stations))
print("Valid indices: 0 ..", len(stations)-1)

payload = {
    "start_idx": 0,          # change these two if you like
    "goal_idx": min(10, len(stations)-1),
    "ev_range_km": 60,       # start generous so a path is likely
    "k_neighbors": 10,       # more graph connectivity
    "assumed_speed_kmh": 60,
    "alpha_weather": 0.15,
    "beta_traffic": 0.10,
    "charge_penalty_min": 15,
    "mode": "FALLBACK"
}

r = requests.post("http://127.0.0.1:8000/route", json=payload)
print("status:", r.status_code)
try:
    print(json.dumps(r.json(), indent=2)[:1200])
except Exception:
    print("Raw response:\n", r.text[:1200])


Stations count: 28
Valid indices: 0 .. 27
status: 500
Raw response:
 Internal Server Error


In [19]:
try:
    proc.terminate()
except:
    pass


In [20]:
%%writefile evat-sprint4/routing_engine/engine.py
import pandas as pd
import numpy as np
from collections import defaultdict
import heapq
from .utils import haversine_km, minmax_norm
from .adapters import get_weather_score, get_traffic_proxy

DEFAULTS = {
    "EV_RANGE_KM": 35.0, "K_NEIGHBORS": 6, "ASSUMED_SPEED_KMH": 60.0,
    "ALPHA_WEATHER": 0.15, "BETA_TRAFFIC": 0.10, "CHARGE_TIME_MIN": 15.0,
    "MODE": "FALLBACK",
}

REQ_COLS = {
    "id": ["InfrastructureID","SiteID","StationID","ID","id"],
    "lat": ["Latitude","lat","LAT","Lat"],
    "lon": ["Longitude","lon","LON","Long","Lng"]
}

def pick(df, names):
    for n in names:
        if n in df.columns: return n
    return None

def load_dataset(path):
    df = pd.read_csv(path)

    # Weather score (if missing)
    if "Weather_Sensitivity_Score" not in df.columns:
        for c in ("TMAX","TMIN","PRCP"):
            if c not in df.columns: df[c] = 0.0
        df["Temp_Range"] = df.get("Temp_Range", df["TMAX"] - df["TMIN"])
        df["High_Temp_Flag"] = (pd.to_numeric(df["TMAX"], errors="coerce") > 35).astype(int)
        df["Low_Temp_Flag"]  = (pd.to_numeric(df["TMIN"], errors="coerce") < 5).astype(int)
        df["No_Precip_Flag"] = (pd.to_numeric(df["PRCP"], errors="coerce") == 0).astype(int)

        def tr_score(x):
            if pd.isna(x): return 0.5
            if x > 20: return 1.0
            if x < 10: return 0.0
            return 0.5

        df["Temp_Range_Score"] = pd.to_numeric(df["Temp_Range"], errors="coerce").apply(tr_score)
        df["Weather_Sensitivity_Score"] = (
            2.0*df["High_Temp_Flag"] + 1.5*df["Low_Temp_Flag"] + (1 - df["No_Precip_Flag"]) + df["Temp_Range_Score"]
        )
    df["Weather_Score_Norm"] = np.clip(pd.to_numeric(df["Weather_Sensitivity_Score"], errors="coerce")/5.0,0,1)

    # Traffic proxy (robust)
    if "Traffic_Proxy" not in df.columns:
        if "Congestion_Factor" in df.columns:
            df["Traffic_Proxy"] = pd.to_numeric(df["Congestion_Factor"], errors="coerce")
        elif "Traffic_Volume" in df.columns and "Traffic_Length" in df.columns:
            vol = pd.to_numeric(df["Traffic_Volume"], errors="coerce")
            seg = pd.to_numeric(df["Traffic_Length"], errors="coerce").replace(0, np.nan)
            df["Traffic_Proxy"] = vol / seg
        else:
            df["Traffic_Proxy"] = 0.0
    df["Traffic_Proxy_Norm"] = minmax_norm(pd.to_numeric(df["Traffic_Proxy"], errors="coerce"))
    return df

def build_nodes(df):
    id_col  = pick(df, REQ_COLS["id"])
    lat_col = pick(df, REQ_COLS["lat"])
    lon_col = pick(df, REQ_COLS["lon"])
    if not lat_col or not lon_col:
        raise ValueError("Latitude/Longitude not found in dataset.")

    # If there is no usable ID column, create a synthetic one
    if id_col is None:
        df = df.copy()
        df["__synthetic_id__"] = df.index.astype(str)
        id_col = "__synthetic_id__"

    cols = [id_col, lat_col, lon_col, "Weather_Score_Norm", "Traffic_Proxy_Norm"]
    nodes = df[cols].dropna().copy()
    nodes = nodes.drop_duplicates(subset=[id_col]).reset_index(drop=True)
    nodes.rename(columns={id_col:"node_id", lat_col:"lat", lon_col:"lon"}, inplace=True)
    return nodes

def build_edges(nodes, K_NEIGHBORS, EV_RANGE_KM):
    coords = nodes[["lat","lon"]].to_numpy()
    edges = []
    for i, (la, lo) in enumerate(coords):
        d = haversine_km(la, lo, coords[:,0], coords[:,1])
        idx = np.argsort(d)[1:K_NEIGHBORS+1]
        for j in idx:
            dist_km = float(d[j])
            if dist_km <= EV_RANGE_KM:
                edges.append((i, j, dist_km))
    return edges

def base_time_minutes(dist_km, speed_kmh):
    return 60.0 * dist_km / max(speed_kmh, 1e-6)

def build_costs(nodes, edges, speed_kmh, alpha, beta, mode="FALLBACK"):
    costs = []
    for (u, v, dist_km) in edges:
        travel_min = base_time_minutes(dist_km, speed_kmh)
        w = get_weather_score(nodes.loc[v,"lat"], nodes.loc[v,"lon"], mode=mode,
                              default=float(nodes.loc[v,"Weather_Score_Norm"]))
        t = get_traffic_proxy(nodes.loc[v,"lat"], nodes.loc[v,"lon"], mode=mode,
                              default=float(nodes.loc[v,"Traffic_Proxy_Norm"]))
        cost_min = travel_min * (1 + alpha*w + beta*t)
        costs.append((u, v, dist_km, travel_min, cost_min))
    return costs

def dijkstra_with_stop_penalty(nodes, costs, start_idx, goal_idx, charge_penalty_min):
    from collections import defaultdict
    import heapq

    N = len(nodes)
    if not (0 <= start_idx < N and 0 <= goal_idx < N):
        return None, float("inf")

    adj = defaultdict(list)
    for (u, v, dist_km, travel_min, cost_min) in costs:
        adj[u].append((v, cost_min, dist_km, travel_min))

    dist = [float('inf')]*N
    prev = [-1]*N
    dist[start_idx] = 0.0
    pq = [(0.0, start_idx)]
    visited = set()

    while pq:
        cur_cost, u = heapq.heappop(pq)
        if u in visited:
            continue
        visited.add(u)
        if u == goal_idx:
            break
        for v, edge_cost, _dkm, _tmin in adj[u]:
            add = charge_penalty_min if v != goal_idx else 0.0
            new_cost = cur_cost + edge_cost + add
            if new_cost < dist[v]:
                dist[v] = new_cost
                prev[v] = u
                heapq.heappush(pq, (new_cost, v))

    if dist[goal_idx] == float('inf'):
        return None, float('inf')

    path = []
    x = goal_idx
    while x != -1:
        path.append(x)
        x = prev[x]
    return path[::-1], dist[goal_idx]

def summarize_route(nodes, path):
    if not path: return []
    out = []
    for i, n in enumerate(path):
        out.append({
            "step": i,
            "node_index": int(n),
            "node_id": nodes.loc[n,"node_id"],
            "lat": float(nodes.loc[n,"lat"]),
            "lon": float(nodes.loc[n,"lon"]),
        })
    return out

def _safe_total_cost(val):
    try:
        if val is None or val == float("inf") or val != val:
            return None
        return float(val)
    except Exception:
        return None

def plan_route(
    data_path="data/combined_data.csv",
    start_idx=0, goal_idx=10,
    EV_RANGE_KM=DEFAULTS["EV_RANGE_KM"], K_NEIGHBORS=DEFAULTS["K_NEIGHBORS"],
    ASSUMED_SPEED_KMH=DEFAULTS["ASSUMED_SPEED_KMH"],
    ALPHA_WEATHER=DEFAULTS["ALPHA_WEATHER"], BETA_TRAFFIC=DEFAULTS["BETA_TRAFFIC"],
    CHARGE_TIME_MIN=DEFAULTS["CHARGE_TIME_MIN"], MODE=DEFAULTS["MODE"]
):
    df = load_dataset(data_path)
    nodes = build_nodes(df)
    edges = build_edges(nodes, K_NEIGHBORS, EV_RANGE_KM)
    costs = build_costs(nodes, edges, ASSUMED_SPEED_KMH, ALPHA_WEATHER, BETA_TRAFFIC, mode=MODE)
    path, total_cost = dijkstra_with_stop_penalty(nodes, costs, start_idx, goal_idx, CHARGE_TIME_MIN)

    status = "ok"
    hint = None
    if total_cost == float("inf") or not path:
        status = "unreachable"
        hint = "Increase ev_range_km or k_neighbors; or choose closer indices."

    return {
        "status": status,
        "hint": hint,
        "params": dict(EV_RANGE_KM=EV_RANGE_KM, K_NEIGHBORS=K_NEIGHBORS,
                       ASSUMED_SPEED_KMH=ASSUMED_SPEED_KMH,
                       ALPHA_WEATHER=ALPHA_WEATHER, BETA_TRAFFIC=BETA_TRAFFIC,
                       CHARGE_TIME_MIN=CHARGE_TIME_MIN, MODE=MODE),
        "nodes_count": len(nodes),
        "edges_count": len(edges),
        "path": summarize_route(nodes, path) if path else [],
        "total_cost_min": _safe_total_cost(total_cost),
    }


Overwriting evat-sprint4/routing_engine/engine.py


In [21]:
import subprocess, sys, time
proc = subprocess.Popen(
    [sys.executable, "-m", "uvicorn", "api.main:app", "--host", "127.0.0.1", "--port", "8000"],
    cwd="evat-sprint4",
)
time.sleep(2)
print("API restarted.")


API restarted.


In [23]:
# stop old server if running
try: proc.terminate()
except: pass

import subprocess, sys, time
proc = subprocess.Popen(
    [sys.executable, "-m", "uvicorn", "api.main:app", "--host", "127.0.0.1", "--port", "8000"],
    cwd="evat-sprint4",
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
)
time.sleep(2)

# print a few log lines so we can see errors on the next call
print(proc.stdout.readline().strip())


INFO:     Started server process [46400]


In [24]:
import requests
payload = {
    "start_idx": 0, "goal_idx": 10,
    "ev_range_km": 70, "k_neighbors": 12,
    "assumed_speed_kmh": 60,
    "alpha_weather": 0.15, "beta_traffic": 0.10,
    "charge_penalty_min": 15, "mode": "FALLBACK"
}
r = requests.post("http://127.0.0.1:8000/route", json=payload)
print("status:", r.status_code)
print("raw:", r.text[:800])

# also print a few server log lines
import itertools
print("\n--- server logs ---")
for _ in itertools.islice(proc.stdout, 10):
    print(_.rstrip())


status: 500
raw: Internal Server Error

--- server logs ---
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     127.0.0.1:65447 - "POST /route HTTP/1.1" 500 Internal Server Error
ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\fastapi\encoders.py", line 324, in jsonable_encoder
    data = dict(obj)
TypeError: 'numpy.int64' object is not iterable



In [25]:
%%writefile evat-sprint4/api/main.py
from fastapi import FastAPI
from fastapi.responses import JSONResponse
from pydantic import BaseModel
from routing_engine.engine import plan_route, load_dataset, build_nodes

import numpy as np
import pandas as pd

app = FastAPI(title="EVAT Routing API")
DATA_PATH = "data/combined_data.csv"
_df = load_dataset(DATA_PATH)
_nodes = build_nodes(_df)

class RouteRequest(BaseModel):
    start_idx: int = 0
    goal_idx: int = 10
    ev_range_km: float = 35.0
    k_neighbors: int = 6
    assumed_speed_kmh: float = 60.0
    alpha_weather: float = 0.15
    beta_traffic: float = 0.10
    charge_penalty_min: float = 15.0
    mode: str = "FALLBACK"

def to_native(x):
    # Convert anything numpy/pandas into plain Python JSON-safe values
    if isinstance(x, (np.integer,)):
        return int(x)
    if isinstance(x, (np.floating,)):
        v = float(x)
        if not np.isfinite(v):  # inf/NaN -> None
            return None
        return v
    if isinstance(x, (np.ndarray,)):
        return [to_native(v) for v in x.tolist()]
    if isinstance(x, pd.Series):
        return [to_native(v) for v in x.to_list()]
    if isinstance(x, pd.DataFrame):
        return [to_native(r) for r in x.to_dict(orient="records")]
    if isinstance(x, dict):
        return {k: to_native(v) for k, v in x.items()}
    if isinstance(x, (list, tuple)):
        return [to_native(v) for v in x]
    return x  # plain python (int/float/str/bool/None)

@app.get("/health")
def health():
    return {"status": "ok", "nodes": int(len(_nodes))}

@app.get("/stations")
def stations():
    return to_native(_nodes)

@app.post("/route")
def route(req: RouteRequest):
    try:
        result = plan_route(
            data_path=DATA_PATH,
            start_idx=req.start_idx, goal_idx=req.goal_idx,
            EV_RANGE_KM=req.ev_range_km, K_NEIGHBORS=req.k_neighbors,
            ASSUMED_SPEED_KMH=req.assumed_speed_kmh,
            ALPHA_WEATHER=req.alpha_weather, BETA_TRAFFIC=req.beta_traffic,
            CHARGE_TIME_MIN=req.charge_penalty_min, MODE=req.mode
        )
        return JSONResponse(content=to_native(result))
    except Exception as e:
        return JSONResponse(status_code=500, content={"status":"error","detail":str(e)})


Overwriting evat-sprint4/api/main.py


In [26]:
try:
    proc.terminate()
except:
    pass

import subprocess, sys, time
proc = subprocess.Popen(
    [sys.executable, "-m", "uvicorn", "api.main:app", "--host", "127.0.0.1", "--port", "8000"],
    cwd="evat-sprint4",
)
time.sleep(2)
print("API restarted at http://127.0.0.1:8000")


API restarted at http://127.0.0.1:8000


In [27]:
import requests, json
stations = requests.get("http://127.0.0.1:8000/stations").json()
payload = {
    "start_idx": 0,
    "goal_idx": min(10, len(stations)-1),
    "ev_range_km": 70,
    "k_neighbors": 12,
    "assumed_speed_kmh": 60,
    "alpha_weather": 0.15,
    "beta_traffic": 0.10,
    "charge_penalty_min": 15,
    "mode": "FALLBACK"
}
r = requests.post("http://127.0.0.1:8000/route", json=payload)
print("status:", r.status_code)
print(json.dumps(r.json(), indent=2)[:1200])


status: 200
{
  "status": "ok",
  "hint": null,
  "params": {
    "EV_RANGE_KM": 70.0,
    "K_NEIGHBORS": 12,
    "ASSUMED_SPEED_KMH": 60.0,
    "ALPHA_WEATHER": 0.15,
    "BETA_TRAFFIC": 0.1,
    "CHARGE_TIME_MIN": 15.0,
    "MODE": "FALLBACK"
  },
  "nodes_count": 28,
  "edges_count": 336,
  "path": [
    {
      "step": 0,
      "node_index": 0,
      "node_id": 11,
      "lat": 48.42459974688293,
      "lon": -123.36109330080023
    },
    {
      "step": 1,
      "node_index": 10,
      "node_id": 8,
      "lat": 48.42511383587826,
      "lon": -123.36895447268292
    }
  ],
  "total_cost_min": 0.5916230301594472
}


In [29]:
%pip install folium


  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.2.2 requires pyqt5<5.13, which is not installed.
spyder 5.2.2 requires pyqtwebengine<5.13, which is not installed.
jupyter-server 1.18.1 requires anyio<4,>=3.1.0, but you have anyio 4.10.0 which is incompatible.


In [3]:
try:
    proc.terminate()
except:
    pass


In [4]:
import subprocess, sys, time
proc = subprocess.Popen(
    [sys.executable, "-m", "uvicorn", "api.main:app", "--host", "127.0.0.1", "--port", "8000"],
    cwd="evat-sprint4",
)
time.sleep(2)
print("API running at http://127.0.0.1:8000")


API running at http://127.0.0.1:8000


In [5]:
import requests
print(requests.get("http://127.0.0.1:8000/health").json())
print("stations:", len(requests.get("http://127.0.0.1:8000/stations").json()))


{'status': 'ok', 'nodes': 28}
stations: 28


In [6]:
import folium, requests

payload = {
    "start_idx": 0, "goal_idx": 10,
    "ev_range_km": 70, "k_neighbors": 12,
    "assumed_speed_kmh": 60,
    "alpha_weather": 0.15, "beta_traffic": 0.10,
    "charge_penalty_min": 15, "mode": "FALLBACK"
}
res = requests.post("http://127.0.0.1:8000/route", json=payload).json()

path = res.get("path", [])
if not path:
    print("No path in response. Try increasing ev_range_km or k_neighbors.")
else:
    m = folium.Map(location=[path[0]["lat"], path[0]["lon"]], zoom_start=13)
    coords = []
    for step in path:
        coords.append((step["lat"], step["lon"]))
        folium.Marker(
            (step["lat"], step["lon"]),
            tooltip=f"Step {step['step']} • Node {step['node_id']}"
        ).add_to(m)
    folium.PolyLine(coords, weight=5).add_to(m)
    m  # displays in notebook

    # (optional) save to HTML
    m.save("evat_route_preview.html")
    print("Saved map to evat_route_preview.html")


Saved map to evat_route_preview.html


In [2]:
import requests

BASE = "http://127.0.0.1:8010"  

# health check
print(requests.get(f"{BASE}/health").json())
print("stations:", len(requests.get(f"{BASE}/stations").json()))


{'status': 'ok', 'nodes': 28}
stations: 28


In [3]:
import requests, folium, json

payload = {
    "start_idx": 0, "goal_idx": 10,
    "ev_range_km": 70, "k_neighbors": 12,
    "assumed_speed_kmh": 60,
    "alpha_weather": 0.15, "beta_traffic": 0.10,
    "charge_penalty_min": 15, "mode": "FALLBACK"
}

res = requests.post(f"{BASE}/route", json=payload).json()
print(json.dumps(res, indent=2)[:800])

path = res.get("path", [])
if not path:
    print("No path. Try higher ev_range_km or k_neighbors.")
else:
    m = folium.Map(location=[path[0]["lat"], path[0]["lon"]], zoom_start=13)
    coords = []
    for step in path:
        coords.append((step["lat"], step["lon"]))
        folium.Marker(
            (step["lat"], step["lon"]),
            tooltip=f"Step {step['step']} • Node {step['node_id']}"
        ).add_to(m)
    folium.PolyLine(coords, weight=5).add_to(m)
    m.save("evat_route_preview4.html")
    print("Saved map to evat_route_preview4.html")


{
  "status": "ok",
  "hint": null,
  "params": {
    "EV_RANGE_KM": 70.0,
    "K_NEIGHBORS": 12,
    "ASSUMED_SPEED_KMH": 60.0,
    "ALPHA_WEATHER": 0.15,
    "BETA_TRAFFIC": 0.1,
    "CHARGE_TIME_MIN": 15.0,
    "MODE": "FALLBACK"
  },
  "nodes_count": 28,
  "edges_count": 336,
  "path": [
    {
      "step": 0,
      "node_index": 0,
      "node_id": 11,
      "lat": 48.42459974688293,
      "lon": -123.36109330080023
    },
    {
      "step": 1,
      "node_index": 10,
      "node_id": 8,
      "lat": 48.42511383587826,
      "lon": -123.36895447268292
    }
  ],
  "total_cost_min": 0.5916230301594472
}
Saved map to evat_route_preview4.html


In [4]:
pip install streamlit folium requests



   ---------------------------------------- 10.0/10.0 MB 12.3 MB/s  0:00:00
   ---------------------------------------- 731.2/731.2 kB 10.0 MB/s  0:00:00
   ---------------------------------------- 6.9/6.9 MB 11.8 MB/s  0:00:00
   ---------------------------------------- 26.2/26.2 MB 12.0 MB/s  0:00:02
  Attempting uninstall: tenacity
    Found existing installation: tenacity 8.0.1
    Uninstalling tenacity-8.0.1:
      Successfully uninstalled tenacity-8.0.1



In [5]:
import streamlit as st, requests, folium
from streamlit.components.v1 import html

BASE = "http://127.0.0.1:8010"  

st.title("EVAT • Weather-Aware Routing (Sprint 4)")

# fetch stations once
stations = requests.get(f"{BASE}/stations").json()
N = len(stations)
st.caption(f"{N} stations loaded")

col1, col2 = st.columns(2)
start_idx = col1.number_input("Start index", 0, N-1, 0)
goal_idx  = col2.number_input("Goal index", 0, N-1, min(10, N-1))

ev_range   = st.slider("EV range (km)", 20, 120, 70)
k_neigh    = st.slider("K neighbors", 4, 15, 12)
alpha_w    = st.slider("Weather weight", 0.0, 0.5, 0.15, 0.05)
beta_t     = st.slider("Traffic weight", 0.0, 0.5, 0.10, 0.05)
charge_pen = st.slider("Charge penalty (min)", 0, 40, 15)

if st.button("Plan route"):
    payload = dict(
        start_idx=start_idx, goal_idx=goal_idx,
        ev_range_km=ev_range, k_neighbors=k_neigh,
        assumed_speed_kmh=60.0,
        alpha_weather=alpha_w, beta_traffic=beta_t,
        charge_penalty_min=charge_pen, mode="FALLBACK"
    )
    r = requests.post(f"{BASE}/route", json=payload).json()
    st.write("Status:", r.get("status"), "| Hint:", r.get("hint"))
    st.json(r["params"])

    path = r.get("path", [])
    if not path:
        st.warning("No path. Try increasing EV range or K neighbors.")
    else:
        m = folium.Map(location=[path[0]["lat"], path[0]["lon"]], zoom_start=13)
        coords = []
        for step in path:
            coords.append((step["lat"], step["lon"]))
            folium.Marker((step["lat"], step["lon"]),
                          tooltip=f"Step {step['step']} • Node {step['node_id']}").add_to(m)
        folium.PolyLine(coords, weight=5).add_to(m)
        folium.Marker(coords[-1], tooltip=f"Goal • Cost (min): {r.get('total_cost_min')}").add_to(m)
        html(m._repr_html_(), height=520)


2025-09-18 20:53:25.947 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-18 20:53:26.431 
  command:

    streamlit run C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-09-18 20:53:26.431 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-18 20:53:26.431 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-18 20:53:26.466 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-18 20:53:26.467 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-18 20:53:26.467 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-18 20:53:26.468 Thread 'MainThread': mi

In [6]:
%%writefile C:/Users/Uday/evat-sprint4/ui_app.py
import streamlit as st, requests, folium
from streamlit.components.v1 import html

BASE = "http://127.0.0.1:8000"

st.title("EVAT • Weather-Aware Routing (Sprint 4)")

try:
    stations = requests.get(f"{BASE}/stations", timeout=5).json()
    N = len(stations)
    st.caption(f"{N} stations loaded from API")
except Exception as e:
    st.error(f"Could not reach API at {BASE}. Start FastAPI first. Details: {e}")
    st.stop()

col1, col2 = st.columns(2)
start_idx = col1.number_input("Start index", 0, N-1, 0)
goal_idx  = col2.number_input("Goal index", 0, N-1, min(10, N-1))

ev_range   = st.slider("EV range (km)", 20, 120, 70)
k_neigh    = st.slider("K neighbors", 4, 15, 12)
alpha_w    = st.slider("Weather weight", 0.0, 0.5, 0.15, 0.05)
beta_t     = st.slider("Traffic weight", 0.0, 0.5, 0.10, 0.05)
charge_pen = st.slider("Charge penalty (min)", 0, 40, 15)

if st.button("Plan route"):
    payload = dict(
        start_idx=int(start_idx), goal_idx=int(goal_idx),
        ev_range_km=float(ev_range), k_neighbors=int(k_neigh),
        assumed_speed_kmh=60.0,
        alpha_weather=float(alpha_w), beta_traffic=float(beta_t),
        charge_penalty_min=float(charge_pen), mode="FALLBACK"
    )
    try:
        r = requests.post(f"{BASE}/route", json=payload, timeout=10)
        data = r.json()
    except Exception as e:
        st.error(f"Request failed: {e}")
        st.stop()

    st.write("Status:", data.get("status"), "| Hint:", data.get("hint"))
    st.json(data.get("params", {}))

    path = data.get("path", [])
    if not path:
        st.warning("No path. Try increasing EV range or K neighbors.")
    else:
        m = folium.Map(location=[path[0]["lat"], path[0]["lon"]], zoom_start=13)
        coords = []
        for step in path:
            coords.append((step["lat"], step["lon"]))
            folium.Marker(
                (step["lat"], step["lon"]),
                tooltip=f"Step {step['step']} • Node {step['node_id']}"
            ).add_to(m)
        folium.PolyLine(coords, weight=5).add_to(m)
        folium.Marker(coords[-1],
                      tooltip=f"Goal • Cost (min): {data.get('total_cost_min')}").add_to(m)
        html(m._repr_html_(), height=520)


Writing C:/Users/Uday/evat-sprint4/ui_app.py


In [7]:
pip install streamlit folium requests


Note: you may need to restart the kernel to use updated packages.


In [ ]:
!cd C:\Users\Uday\evat-sprint4 && streamlit run ui_app.py
